**AUTOML**  


Here we perform two popular auto-ml algorithms in order to verify if they give us better results compared to traditional methods.


In [1]:
import pandas as pd

In [3]:
train = pd.read_csv('../data/data/model_data/train.csv')

In [4]:
test = pd.read_csv('../data/data/model_data/test.csv')

**AutoGluon**

In [20]:
from autogluon.tabular import TabularPredictor

In [6]:
predictor = TabularPredictor(label='Mean_Radiation').fit(train)

No path specified. Models will be saved in: "AutogluonModels/ag-20241207_212159"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.0.0: Tue Sep 24 23:38:45 PDT 2024; root:xnu-11215.1.12~1/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       3.92 GB / 16.00 GB (24.5%)
Disk Space Avail:   815.20 GB / 926.35 GB (88.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Defau

[1000]	valid_set's rmse: 78.3686


	-78.2743	 = Validation score   (-root_mean_squared_error)
	3.76s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	-81.0638	 = Validation score   (-root_mean_squared_error)
	1.09s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-88.9357	 = Validation score   (-root_mean_squared_error)
	0.98s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: CatBoost ...
	-75.5036	 = Validation score   (-root_mean_squared_error)
	1.62s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-88.6116	 = Validation score   (-root_mean_squared_error)
	0.28s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
		Import fastai failed. A quick tip is to install via `pip install autogluon.tabular[fastai]==1.1.1`. 
Fitting model: XGBoost ...
	-81.3329	 = Validation score   (-root_mean_squared_error)
	0.68s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting mode

In [10]:
predicted = predictor.predict(test)
true = test['Mean_Radiation']
MSE = ((predicted - true) ** 2).mean()
print(f'MSE: {MSE}')
Rsq = 1 - MSE / true.var()
print(f'R^2: {Rsq}')
Rsq_adj = 1 - (1 - Rsq) * (len(true) - 1) / (len(true) - test.shape[1] - 1)
print(f'Adjusted R^2: {Rsq_adj}')
MAE = (abs(predicted - true)).mean()
print(f'MAE: {MAE}')

MSE: 5213.0185313055645
R^2: 0.9473648931488053
Adjusted R^2: 0.9446733251848238
MAE: 37.59544307256026


The R^2 result seems promising.

In [8]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-69.881171,-75.503590,root_mean_squared_error,0.006527,0.001596,1.623620,0.006527,0.001596,1.623620,1,True,6
1,LightGBM,-70.560555,-81.063827,root_mean_squared_error,0.003463,0.001650,1.086363,0.003463,0.001650,1.086363,1,True,4
2,WeightedEnsemble_L2,-72.201236,-72.673405,root_mean_squared_error,0.041878,0.019890,11.547472,0.001463,0.000156,0.005290,2,True,11
3,XGBoost,-72.724312,-81.332889,root_mean_squared_error,0.012251,0.003116,0.675948,0.012251,0.003116,0.675948,1,True,8
4,LightGBMXT,-74.129227,-78.274325,root_mean_squared_error,0.014568,0.010481,3.762534,0.014568,0.010481,3.762534,1,True,3
5,RandomForestMSE,-74.541995,-88.935728,root_mean_squared_error,0.051749,0.027321,0.981014,0.051749,0.027321,0.981014,1,True,5
6,LightGBMLarge,-75.297023,-88.680528,root_mean_squared_error,0.013860,0.006112,6.586896,0.013860,0.006112,6.586896,1,True,10
7,ExtraTreesMSE,-77.063867,-88.611632,root_mean_squared_error,0.047748,0.025711,0.276558,0.047748,0.025711,0.276558,1,True,7
8,NeuralNetTorch,-99.516941,-85.276269,root_mean_squared_error,0.007069,0.004540,5.480080,0.007069,0.004540,5.480080,1,True,9
9,KNeighborsDist,-336.680691,-334.778468,root_mean_squared_error,0.003873,0.001549,0.004026,0.003873,0.001549,0.004026,1,True,2
